This notebook aims at showing the ways of including objects into the mockup.

First thing first, a few considéreations about DART functionning with objects. DART considers the object to be in a X forward, Y up system. In other words, the vertex coordintaes must be written in the following order inside the .obj file:
```
v y z x
```

The code below creates an obj file with a rectangle of coordinates in meters $x_{min}=2$, $y_{min}=1$, $x_{max}=5.5$, $y_{max}=4$

In [ ]:
import os
import glob
import matplotlib.pyplot as plt
import pytools4dart as ptd

obj_str = '''
v 4.0 3.0 2.0
v 4.0 3.0 5.5
v 1.0 3.0 5.5
v 1.0 3.0 2.0
f 4 1 2
f 4 2 3
'''
objFpath = os.path.abspath('data/rectangle.obj')
with open(objFpath, mode='w') as f:
    f.write(obj_str)

The code to produce a simple RGB simulation with that rectangle 3m over the ground is the following.

In [ ]:
simu = ptd.simulation('object_3d', empty=True)

simu.scene.size = [10, 10]
simu.scene.cell = [.1, .1]

# RGB bands
for wvl in [0.485, 0.555, 0.655]:
    simu.add.band(wvl=wvl, bw=0.07)

# ground optical properties
op_ground = {
    'type':'Lambertian',
    'ident':'ground',
    'databaseName':'Lambertian_mineral.db',
    'ModelName':'clay_brown'}
    #  'ModelName':'reflect_equal_1_trans_equal_1_1'}
    

simu.add.optical_property(**op_ground)
simu.scene.ground.OpticalPropertyLink.ident='ground'

# target properties
op_vegetation = {'type':'Lambertian',
              'ident': 'target',
              'databaseName':'Lambertian_vegetation.db',
              'ModelName':'grass_rye'}

op = simu.add.optical_property(**op_vegetation)

# object
obj = os.path.abspath(objFpath)
obj3D = simu.add.object_3d(obj, xpos=0, ypos=0, zpos=0)
obj3D.ObjectOpticalProperties.OpticalPropertyLink.ident = 'target'

# write and run simulation, and make the composite image
simu.write(overwrite=True)
simu.run.full()
rgbDpath = simu.run.colorCompositeBands(red=2, green=1, blue=0, iteration='X', outdir='rgb')



In [ ]:
# show image
rgbDpath = '/home/boissieu/DART_5-7-4_v1083/user_data/simulations/object_3d/output/rgb'
imageFpath = glob.glob(os.path.join(rgbDpath, 'ima01*.png'))[0]
image = plt.imread(imageFpath)
plt.imshow(image)
plt.show()


Opening the files in rgb directory, you will find a red square that is 100x100 pixels. The image is actually showing only the ground. The rectangle target is not shown because of parameter `objectDEMMode=0` <=> 'Put on DEM'. Thus it translates object so that object Zmin=Zground at object Xmin,Ymin.

Set parameter `objectDEMMode=2` <=> 'Ignore DEM' to avoid this behaviour. This way the object is considered as not connected to the ground.


In [ ]:
obj3D.objectDEMMode=2
simu.write(overwrite=True)
simu.run.full()
rgbDpath = simu.run.colorCompositeBands(red=2, green=1, blue=0, iteration='X', outdir='rgb')

imageFpath = glob.glob(os.path.join(rgbDpath, 'ima01*.png'))[0]
image = plt.imread(imageFpath)
plt.imshow(image)
plt.show()


This time the rectangle appears over the ground, however it is black. This is because the normals are wrongly oriented.


One way to avoid this problem is to consider the object as double face, and give the same optical properties to both.


In [ ]:
obj3D.ObjectOpticalProperties.doubleFace=1
obj3D.ObjectOpticalProperties.BackFaceOpticalProperty.OpticalPropertyLink.ident = 'target'

simu.write(overwrite=True)
simu.run.full()
rgbDpath = simu.run.colorCompositeBands(red=2, green=1, blue=0, iteration='X', outdir='rgb')

image = plt.imread(imageFpath)
plt.imshow(image)
plt.show()



Another way is to orient correctly the facets face to the sky. A facet is considered to face the sky when the corners are given in the anti-clockwise order on X,Y plane. This can be specified either in the facet lines, e.g. `f 2 1 4` instead of `f 4 1 2`. However, it can be quite tricky with a large amount of triangles.

In [ ]:
obj_str = '''
v 4.0 3.0 2.0
v 4.0 3.0 5.5
v 1.0 3.0 5.5
v 1.0 3.0 2.0
f 2 1 4
f 3 2 4
'''
with open(objFpath, mode='w') as f:
    f.write(obj_str)

obj3D.ObjectOpticalProperties.doubleFace=0

simu.write(overwrite=True)
simu.run.full()
simu.run.colorCompositeBands(red=2, green=1, blue=0, iteration='X', outdir='rgb')

image = plt.imread(imageFpath)
plt.imshow(image)
plt.show()
